In [1]:
import pandas as pd

REDDIT = r'C:\Users\DELL\Desktop\FinalYear\StressDetection\Reddit_Combi.csv'
USECOLS = ['Body_Title', 'label']
df = pd.read_csv(filepath_or_buffer=REDDIT, sep=';', usecols=USECOLS)
df.head()

,Body_Title,label
0,Envy to other is swallowing me Im from develop...,1
1,Nothin outta the ordinary. Paradise. Job stres...,1
2,Almost 49 and the chasm of emptiness has never...,1
3,I’m happy again After my closest friend left m...,0
4,Is it possible to recover from such a traumati...,1


In [2]:
from plotly import express
express.pie(data_frame=df, names='label', color='label')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[1],
                                   [1],
                                   [1],
                                   ...,
                                   [1],
                                   [1],
                                   [1]], dtype=int64),
              'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hovertemplate': 'label=%{customdata[0]}<extra></extra>',
              'labels': array([1, 1, 1, ..., 1, 1, 1], dtype=int64),
              'legendgroup': '',
              'marker': {'colors': [#636efa, #636efa, #636efa, ..., #636efa,
                                    #636efa, #636efa]},
              'name': '',
              'showlegend': True,
              'type': 'pie'}],
    'layout': {'legend': {'tracegroupgap': 0}, 'margin': {'t': 60}, 'template': '...'}
})

In [3]:
express.histogram(x=df['Body_Title'].str.len(), log_y=True)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'bingroup': 'x',
              'hovertemplate': 'x=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'histogram',
              'x': array([1196, 3902,  554, ...,  287, 1110,  668], dtype=int64),
              'xaxis': 'x',
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'x'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'count'}, 'type': 'log'}}
})

In [4]:
express.histogram(x=df['Body_Title'].str.split().str.len(), log_y=True)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'bingroup': 'x',
              'hovertemplate': 'x=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'histogram',
              'x': array([226, 750, 106, ...,  64, 216, 121], dtype=int64),
              'xaxis': 'x',
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'x'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'count'}, 'type': 'log'}}
})

In [5]:
from gensim.corpora import Dictionary
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_tags
CUSTOM_FILTERS = [lambda x: x.lower(), 
                  remove_stopwords, 
                  strip_multiple_whitespaces, 
                  strip_numeric,
                  strip_punctuation,
                  strip_short,
                  strip_tags, 
                 ]
documents = df['Body_Title'].values.tolist()
texts = [preprocess_string(s=document, filters=CUSTOM_FILTERS) for document in documents]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
print(dictionary)


Dictionary<14372 unique tokens: ['afford', 'age', 'beetwen', 'better', 'big']...>


In [6]:
from arrow import now
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
doc2vec_start = now()
doc2vec_model = Doc2Vec(vector_size=100, min_count=20, epochs=40)
corpus_iterable = [TaggedDocument(item, [index]) for index, item in enumerate(corpus) ]
doc2vec_model.build_vocab(corpus_iterable=corpus_iterable)
doc2vec_model.train(corpus_iterable=corpus_iterable, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs,)
df['vectors'] = doc2vec_model.dv.vectors.tolist()
print('doc2vec training time: {}'.format(now() - doc2vec_start))

doc2vec training time: 0:00:09.956813


In [7]:
from umap import UMAP

doc2vec_umap_start = now()
doc2vec_umap_model = UMAP(n_components=2, random_state=2024, verbose=1, init='pca', n_jobs=1)
df[['x', 'y']] = doc2vec_umap_model.fit_transform(X=df['vectors'].apply(func=pd.Series),)
df['short document'] = df['Body_Title'].str[:80]
print('doc2vec umap time: {}'.format(now() - doc2vec_umap_start))

C:\Users\DELL\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



UMAP(init='pca', n_jobs=1, random_state=2024, verbose=1)
Thu May 16 17:24:56 2024 Construct fuzzy simplicial set
Thu May 16 17:25:03 2024 Finding Nearest Neighbors
Thu May 16 17:25:06 2024 Finished Nearest Neighbor Search
Thu May 16 17:25:09 2024 Construct embedding


Epochs completed:   3%| ▎          13/500 [00:00]

	completed  0  /  500 epochs


Epochs completed:  13%| █▎         63/500 [00:01]

...

	completed  400  /  500 epochs


Epochs completed:  93%| █████████▎ 464/500 [00:05]

	completed  450  /  500 epochs


Epochs completed: 100%| ██████████ 500/500 [00:06]

Thu May 16 17:25:15 2024 Finished embedding
doc2vec umap time: 0:00:19.477637


In [8]:
from gensim.corpora import Dictionary
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_tags
CUSTOM_FILTERS = [lambda x: x.lower(), 
                  remove_stopwords, 
                  strip_multiple_whitespaces, 
                  strip_numeric,
                  strip_punctuation,
                  strip_short,
                  strip_tags, 
                 ]
documents = df['Body_Title'].values.tolist()
texts = [preprocess_string(s=document, filters=CUSTOM_FILTERS) for document in documents]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
print(dictionary)


Dictionary<14372 unique tokens: ['afford', 'age', 'beetwen', 'better', 'big']...>


In [9]:
express.scatter(data_frame=df, x='x', y='y', color='label', height=800, hover_name='short document')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': ('<b>%{hovertext}</b><br><br>x=%' ... '%{marker.color}<extra></extra>'),
              'hovertext': array(['Envy to other is swallowing me Im from developingcountry, Indonesia , and for no',
                                  'Nothin outta the ordinary. Paradise. Job stress. Life stress. (F bombs call me h',
                                  'Almost 49 and the chasm of emptiness has never been greater I’ve been diagnosed ',
                                  ...,
                                  'I feel as if I actually died a long time ago and I am on autopilot now I feel li',
                                  'Is it wierd that i have imaginary friends at the age of 16 (almost 17) ... i fee',
                                  'Head and Eye Pressure Since October (for the last 4 months) I have had the follo'],
                                 dtype=object),
              'legendgroup': '',
              'marker': {'color': array([1, 1, 1, ..., 1, 1, 1], dtype=int64),
                         'coloraxis': 'coloraxis',
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'showlegend': False,
              'type': 'scattergl',
              'x': array([ 8.464807 , 10.725334 ,  6.612285 , ...,  0.6181435,  5.467995 ,
                           6.338467 ], dtype=float32),
              'xaxis': 'x',
              'y': array([5.411109 , 5.9918957, 6.679663 , ..., 7.775756 , 8.435319 , 6.3459854],
                         dtype=float32),
              'yaxis': 'y'}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'label'}},
                             'colorscale': [[0.0, '#0d0887'], [0.1111111111111111,
                                            '#46039f'], [0.2222222222222222,
                                            '#7201a8'], [0.3333333333333333,
                                            '#9c179e'], [0.4444444444444444,
                                            '#bd3786'], [0.5555555555555556,
                                            '#d8576b'], [0.6666666666666666,
                                            '#ed7953'], [0.7777777777777778,
                                            '#fb9f3a'], [0.8888888888888888,
                                            '#fdca26'], [1.0, '#f0f921']]},
               'height': 800,
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'x'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'y'}}}
})

In [10]:
import arrow
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['vectors'].apply(func=pd.Series), df['label'], test_size=0.25, random_state=2024, stratify=df['label'])

time_start = arrow.now()
regression = LogisticRegression(max_iter=100000, tol=1e-12).fit(X=X_train, y=y_train)
print('model fit in {} iterations took {}'.format(regression.n_iter_[0], arrow.now() - time_start))

print('accuracy: {:5.4f}'.format(accuracy_score(y_true=y_test, y_pred=regression.predict(X=X_test))))
print('model done in {}'.format(now() - time_start))

model fit in 98 iterations took 0:00:00.058261
accuracy: 0.8886
model done in 0:00:00.060263


In [11]:
from sklearn.metrics import classification_report

print(classification_report(y_true=y_test, y_pred=regression.predict(X=X_test)))

              precision    recall  f1-score   support

           0       0.79      0.12      0.20        95
           1       0.89      1.00      0.94       686

    accuracy                           0.89       781
   macro avg       0.84      0.56      0.57       781
weighted avg       0.88      0.89      0.85       781



In [12]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

gauss = GaussianProcessClassifier(1.0 * RBF(1.0), random_state=2024)
gauss.fit(X=X_train, y=y_train)

print(classification_report(y_true=y_test, y_pred=gauss.predict(X=X_test)))

              precision    recall  f1-score   support

           0       0.59      0.23      0.33        95
           1       0.90      0.98      0.94       686

    accuracy                           0.89       781
   macro avg       0.75      0.60      0.64       781
weighted avg       0.86      0.89      0.86       781



In [13]:
%env TOKENIZERS_PARALLELISM=false
!pip install --quiet keybert
print('pip install keybert complete.')

env: TOKENIZERS_PARALLELISM=false
pip install keybert complete.


In [14]:
from arrow import now
from keybert import KeyBERT
from sklearn.feature_extraction.text import TfidfVectorizer

MAX_DF = 1.0
MIN_DF = 4
MODEL = 'all-MiniLM-L12-v2'
STOP_WORDS = 'english'
DOCS = df['Body_Title'].values.tolist()

model_start = now()
bert = KeyBERT(model=MODEL,)
bert.max_seq_length = 512
vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words=STOP_WORDS, min_df=MIN_DF, max_df=MAX_DF, )
document_embeddings, word_embeddings = bert.extract_embeddings(docs=DOCS, vectorizer=vectorizer, )
print('embedding time: {}'.format(now() - model_start))
print('we have {} documents and {} words.'.format(len(document_embeddings), len(word_embeddings)))


embedding time: 0:03:25.994718
we have 3123 documents and 4663 words.


In [16]:
df['embedding'] = document_embeddings.tolist()
embedding_umap_model = UMAP(n_components=2, random_state=2024, verbose=1, init='pca', n_jobs=1)
df[['ex', 'ey']] = embedding_umap_model.fit_transform(X=df['embedding'].apply(func=pd.Series),)
express.scatter(data_frame=df, x='ex', y='ey', color='label', height=800, hover_name='short document')

UMAP(init='pca', n_jobs=1, random_state=2024, verbose=1)
Sun May 12 23:29:45 2024 Construct fuzzy simplicial set
Sun May 12 23:29:52 2024 Finding Nearest Neighbors
Sun May 12 23:29:53 2024 Finished Nearest Neighbor Search
Sun May 12 23:29:53 2024 Construct embedding


Epochs completed:   2%| ▏          11/500 [00:00]

	completed  0  /  500 epochs


Epochs completed:  13%| █▎         66/500 [00:01]

	completed  50  /  500 epochs


...

	completed  450  /  500 epochs


Epochs completed: 100%| ██████████ 500/500 [00:07]


Sun May 12 23:30:00 2024 Finished embedding


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': ('<b>%{hovertext}</b><br><br>ex=' ... '%{marker.color}<extra></extra>'),
              'hovertext': array(['Envy to other is swallowing me Im from developingcountry, Indonesia , and for no',
                                  'Nothin outta the ordinary. Paradise. Job stress. Life stress. (F bombs call me h',
                                  'Almost 49 and the chasm of emptiness has never been greater I’ve been diagnosed ',
                                  ...,
                                  'I feel as if I actually died a long time ago and I am on autopilot now I feel li',
                                  'Is it wierd that i have imaginary friends at the age of 16 (almost 17) ... i fee',
                                  'Head and Eye Pressure Since October (for the last 4 months) I have had the follo'],
                                 dtype=object),
              'legendgroup': '',
              'marker': {'color': array([1, 1, 1, ..., 1, 1, 1], dtype=int64),
                         'coloraxis': 'coloraxis',
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'showlegend': False,
              'type': 'scattergl',
              'x': array([5.4913607, 2.470916 , 3.4782128, ..., 2.064542 , 1.4783998, 7.304739 ],
                         dtype=float32),
              'xaxis': 'x',
              'y': array([2.6484914 , 2.0097952 , 2.4858587 , ..., 0.8843907 , 3.9592826 ,
                          0.68019307], dtype=float32),
              'yaxis': 'y'}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'label'}},
                             'colorscale': [[0.0, '#0d0887'], [0.1111111111111111,
                                            '#46039f'], [0.2222222222222222,
                                            '#7201a8'], [0.3333333333333333,
                                            '#9c179e'], [0.4444444444444444,
                                            '#bd3786'], [0.5555555555555556,
                                            '#d8576b'], [0.6666666666666666,
                                            '#ed7953'], [0.7777777777777778,
                                            '#fb9f3a'], [0.8888888888888888,
                                            '#fdca26'], [1.0, '#f0f921']]},
               'height': 800,
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'ex'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'ey'}}}
})

In [17]:
import arrow
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Xe_train, Xe_test, ye_train, ye_test = train_test_split(df['embedding'].apply(func=pd.Series), df['label'], test_size=0.25, random_state=2024, stratify=df['label'])

time_start = arrow.now()
embedding_regression = LogisticRegression(max_iter=100000, tol=1e-12).fit(X=Xe_train, y=ye_train)
print('model fit in {} iterations took {}'.format(embedding_regression.n_iter_[0], arrow.now() - time_start))

print('accuracy: {:5.4f}'.format(accuracy_score(y_true=ye_test, y_pred=embedding_regression.predict(X=Xe_test))))
print('model done in {}'.format(now() - time_start))

model fit in 37 iterations took 0:00:00.072882
accuracy: 0.9449
model done in 0:00:00.078874


In [18]:
print(classification_report(y_true=ye_test, y_pred=embedding_regression.predict(X=Xe_test)))

              precision    recall  f1-score   support

           0       0.92      0.60      0.73        95
           1       0.95      0.99      0.97       686

    accuracy                           0.94       781
   macro avg       0.93      0.80      0.85       781
weighted avg       0.94      0.94      0.94       781



In [19]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(alpha=1.0, max_iter=100000, random_state=2024).fit(X=Xe_train, y=ye_train)
print('score: {:5.4f}'.format(mlp.score(X=Xe_test, y=ye_test)))

print(classification_report(y_true=ye_test, y_pred=mlp.predict(X=Xe_test)))

score: 0.9462
              precision    recall  f1-score   support

           0       0.92      0.61      0.73        95
           1       0.95      0.99      0.97       686

    accuracy                           0.95       781
   macro avg       0.93      0.80      0.85       781
weighted avg       0.95      0.95      0.94       781



In [43]:
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_tags

# Preprocessing function
CUSTOM_FILTERS = [
    lambda x: x.lower(),
    remove_stopwords,
    strip_multiple_whitespaces,
    strip_numeric,
    strip_punctuation,
    strip_short,
    strip_tags,
]

def preprocess_text(text):
    return preprocess_string(text, filters=CUSTOM_FILTERS)

# Example input text
input_text = "I enjoy the peaceful atmosphere of the house I'm house-sitting."

# Preprocess the input text
preprocessed_input = preprocess_text(input_text)

# Generate Doc2Vec embeddings for the preprocessed input text
input_vector = doc2vec_model.infer_vector(preprocessed_input)


In [44]:
# Predict probabilities instead of classes
prediction_proba = regression.predict_proba(input_vector_reshaped)

# Get the probability of being in the "Not Stressful" class
not_stressful_proba = prediction_proba[0][0]

# Define a threshold (e.g., 0.5)
threshold = 0.5

# Make prediction based on the adjusted threshold
predicted_label = "Stressful" if not_stressful_proba < threshold else "Not Stressful"

print("Predicted Label:", predicted_label)


Predicted Label: Stressful


In [48]:
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_tags

# Preprocessing function
CUSTOM_FILTERS = [
    lambda x: x.lower(),
    remove_stopwords,
    strip_multiple_whitespaces,
    strip_numeric,
    strip_punctuation,
    strip_short,
    strip_tags,
]

def preprocess_text(text):
    return preprocess_string(text, filters=CUSTOM_FILTERS)

# Example input text
input_text = "I enjoy the peaceful atmosphere of the house I'm house-sitting."

# Preprocess the input text
preprocessed_input = preprocess_text(input_text)

# Generate Doc2Vec embeddings for the preprocessed input text
input_vector = doc2vec_model.infer_vector(preprocessed_input)

# Reshape input_vector to match the shape of the training data
input_vector_reshaped = input_vector.reshape(1, -1)

# Predict probabilities instead of classes
prediction_proba = regression.predict_proba(input_vector_reshaped)

# Get the probability of being in the "Not Stressful" class
not_stressful_proba = prediction_proba[0][0]

# Define a threshold (e.g., 0.5)
threshold = 0.5

# Make prediction based on the adjusted threshold
predicted_label = "Stressful" if not_stressful_proba < threshold else "Not Stressful"

print("Predicted Label:", predicted_label)


Predicted Label: Stressful


In [50]:
# Lexicon of stress-indicative words
stress_lexicon = {"stress", "anxiety", "pressure", "overwhelmed", "panic", "worry", "tension"}

def detect_stress(input_text):
    # Preprocess input text (optional)
    preprocessed_text = input_text.lower()

    # Initialize total stress score
    total_score = 0

    # Calculate score based on presence of stress-indicative words
    for word in stress_lexicon:
        if word in preprocessed_text:
            # Increment score for each occurrence of stress-indicative word
            total_score += 1

    # Define threshold for classifying text as stressful
    threshold = 0  # Adjust based on the sensitivity desired

    # Classify text based on total stress score
    if total_score >= threshold:
        return "Stressful"
    else:
        return "Not Stressful"

# Example input text
input_text = "I'm feeling a lot of stress lately due to work deadlines."

# Detect stress in the input text
stress_label = detect_stress(input_text)
print("Stress Label:", stress_label)


Stress Label: Not Stressful


In [46]:
# Check if doc2vec_model is trained
if doc2vec_model is not None:
    if hasattr(doc2vec_model, 'dv'):
        print("Doc2Vec model is trained.")
    else:
        print("Doc2Vec model is not trained.")
else:
    print("Doc2Vec model is not initialized.")


Doc2Vec model is trained.


In [47]:
# Check if regression model is trained
if regression is not None:
    if hasattr(regression, 'coef_') and hasattr(regression, 'intercept_'):
        print("Regression model is trained.")
    else:
        print("Regression model is not trained.")
else:
    print("Regression model is not initialized.")


Regression model is trained.
